### RAG con Memoria Persistente usando LangGraph

En este notebook aprenderemos a implementar un sistema RAG (Retrieval-Augmented Generation) que mantiene el historial de conversaciones usando la funcionalidad de memoria persistente de LangGraph.

In [ ]:
# Importamos las librerías necesarias para configuración del entorno
import os  # Para manejar variables de entorno del sistema operativo
from dotenv import load_dotenv  # Para cargar variables desde archivo .env

# Cargamos las variables de entorno desde el archivo .env
load_dotenv()

# Configuramos la API key de OpenAI desde las variables de entorno
# Esto es necesario para autenticarnos con el servicio de OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Importamos la función para inicializar modelos de chat de LangChain
from langchain.chat_models import init_chat_model

# Inicializamos el modelo de lenguaje GPT-5 de OpenAI
# Este será el LLM (Large Language Model) que usaremos para las respuestas
llm = init_chat_model("openai:gpt-5")

# Mostramos la configuración del modelo para verificar que se inicializó correctamente
llm

In [ ]:
# Importamos la clase OpenAIEmbeddings de LangChain
# Los embeddings son representaciones vectoriales de texto que capturan significado semántico
from langchain_openai import OpenAIEmbeddings

# Inicializamos el modelo de embeddings de OpenAI
# Por defecto usa 'text-embedding-ada-002' que convierte texto en vectores de 1536 dimensiones
embeddings = OpenAIEmbeddings()

# Mostramos la configuración del embedding para verificar sus parámetros
embeddings

In [ ]:
## Ingesta y Procesamiento de Documentos

# Importamos BeautifulSoup para parsear HTML
import bs4

# Importamos el cargador de documentos web de LangChain
from langchain_community.document_loaders import WebBaseLoader

# Importamos la clase Document que representa un documento con contenido y metadatos
from langchain_core.documents import Document

# Importamos el splitter de texto recursivo para dividir documentos en chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Importamos tipos para type hints y mejor documentación del código
from typing_extensions import List, TypedDict

In [ ]:
# Cargamos y procesamos el contenido de un blog sobre agentes de IA

# Creamos un loader para extraer contenido web
loader = WebBaseLoader(
    # URL del blog de Lilian Weng sobre agentes de IA
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    
    # Configuramos BeautifulSoup para extraer solo las secciones relevantes
    bs_kwargs=dict(
        # SoupStrainer filtra el HTML para extraer solo elementos específicos
        parse_only=bs4.SoupStrainer(
            # Extraemos solo elementos con estas clases CSS
            # Esto nos da el título, encabezado y contenido del post, ignorando navegación/footer
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

# Ejecutamos el loader para descargar y parsear el contenido
# docs será una lista de objetos Document con page_content y metadata
docs = loader.load()

# Mostramos los documentos cargados para verificar el contenido
docs

In [ ]:
## Chunking (División en fragmentos)

# Creamos un splitter de texto recursivo para dividir documentos largos en chunks manejables
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # Tamaño máximo de cada chunk en caracteres
    chunk_overlap=200       # Superposición entre chunks para mantener contexto entre fragmentos
)

# Dividimos todos los documentos cargados en chunks más pequeños
# Esto es crucial para RAG porque:
# 1. Los modelos tienen límites de contexto
# 2. Chunks más pequeños mejoran la precisión de la búsqueda semántica
# 3. Permite recuperar solo las partes relevantes del documento
all_splits = text_splitter.split_documents(docs)

# Mostramos los chunks resultantes
all_splits

In [ ]:
## Vector Store (Almacén de Vectores)

# Importamos FAISS, una biblioteca de Facebook para búsqueda de similitud eficiente
from langchain_community.vectorstores import FAISS

# Creamos un vector store FAISS a partir de nuestros documentos fragmentados
vector_store = FAISS.from_documents(
    documents=all_splits,    # Los chunks de documentos que creamos anteriormente
    embedding=embeddings     # El modelo de embeddings de OpenAI para convertir texto a vectores
)

# FAISS crea un índice optimizado para búsqueda de similitud por coseno
# Cada documento se convierte en un vector de 1536 dimensiones
# El índice permite búsquedas rápidas de los vectores más similares a una consulta

# Mostramos cuántos vectores se almacenaron en el índice
print(f"Vector store creado con {vector_store.index.ntotal} vectores")

In [ ]:
# Importamos el decorador tool de LangChain para crear herramientas personalizadas
# Las herramientas son funciones que los agentes pueden invocar
from langchain.agents import tool

In [ ]:
# Definimos una herramienta de recuperación usando el decorador @tool
@tool()
def retrieve(query: str):
    """Recupera información relacionada con la consulta del usuario"""
    
    # Buscamos los 2 documentos más similares a la consulta en el vector store
    # similarity_search utiliza similitud de coseno entre embeddings
    retrieved_docs = vector_store.similarity_search(query, k=2)
    
    # Serializamos los documentos recuperados en un formato legible
    # Incluimos tanto los metadatos (fuente) como el contenido de cada documento
    serialized = "\n\n".join(
        (f"Fuente: {doc.metadata}\nContenido: {doc.page_content}")
        for doc in retrieved_docs
    )
    
    # Retornamos tanto la versión serializada (para el LLM) como los documentos originales
    return serialized, retrieved_docs

In [ ]:
# Importamos los componentes necesarios de LangGraph para construir el flujo del agente

# SystemMessage: para crear mensajes del sistema que guían el comportamiento del LLM
from langchain_core.messages import SystemMessage

# MemorySaver: gestor de checkpoints que persiste el estado de la conversación
from langgraph.checkpoint.memory import MemorySaver

# END: constante que indica el final del grafo
# MessagesState: estado que mantiene la lista de mensajes de la conversación
# StateGraph: clase para construir grafos de estado
from langgraph.graph import END, MessagesState, StateGraph

# ToolNode: nodo que ejecuta herramientas
# tools_condition: función que decide si llamar herramientas o terminar
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
# Paso 1: Generar un mensaje de IA que puede incluir una llamada a herramienta

def query_or_respond(state: MessagesState):
    """Genera una llamada a herramienta para recuperación o responde directamente."""
    
    # Vinculamos las herramientas al LLM
    # Esto permite que el modelo decida cuándo invocar la herramienta 'retrieve'
    llm_with_tools = llm.bind_tools([retrieve])
    
    # Invocamos el LLM con el historial de mensajes del estado
    # El modelo analiza la conversación y decide si necesita recuperar información
    response = llm_with_tools.invoke(state["messages"])
    
    # MessagesState añade mensajes al estado en lugar de sobrescribirlos
    # Esto mantiene el historial completo de la conversación
    return {"messages": [response]}

In [ ]:
# Paso 2: Ejecutar la recuperación de información

# Creamos un nodo de herramientas que puede ejecutar nuestra función 'retrieve'
# ToolNode es una clase preconfigurada que:
# 1. Extrae las llamadas a herramientas del mensaje de IA
# 2. Ejecuta las herramientas correspondientes
# 3. Retorna los resultados como mensajes de herramienta (ToolMessage)
tools = ToolNode([retrieve])

# Mostramos la configuración del nodo de herramientas
tools

In [ ]:
# Paso 3: Generar una respuesta usando el contenido recuperado

def generate(state: MessagesState):
    """Genera una respuesta basada en el contexto recuperado."""
    
    # Obtenemos los mensajes de herramienta más recientes
    # Recorremos los mensajes en orden inverso para obtener los resultados más recientes
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        # Si el mensaje es de tipo "tool" (resultado de herramienta), lo guardamos
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            # Dejamos de buscar cuando encontramos un mensaje que no es de herramienta
            break
    
    # Revertimos el orden para tener los mensajes en orden cronológico
    tool_messages = recent_tool_messages[::-1]

    # Formateamos el contenido recuperado en un formato legible
    # Concatenamos el contenido de todos los mensajes de herramienta
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    
    # Creamos el mensaje del sistema con el contexto recuperado
    # Este mensaje instruye al LLM sobre cómo usar el contexto
    system_message_content = (
        "Eres un asistente para tareas de respuesta a preguntas. "
        "Usa los siguientes fragmentos de contexto recuperado para responder "
        "la pregunta. Si no sabes la respuesta, di que no lo sabes. "
        "Usa un máximo de tres oraciones y mantén la respuesta concisa."
        "\n\n"
        f"{docs_content}"
    )
    
    # Filtramos los mensajes de conversación para incluir solo:
    # - Mensajes humanos (preguntas del usuario)
    # - Mensajes del sistema
    # - Mensajes de IA que no contienen llamadas a herramientas
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    
    # Construimos el prompt completo: mensaje del sistema + historial de conversación
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Invocamos el LLM con el prompt completo para generar la respuesta final
    response = llm.invoke(prompt)
    
    # Retornamos la respuesta como un nuevo mensaje en el estado
    return {"messages": [response]}

In [ ]:
# Construimos el grafo de estado que orquesta el flujo RAG

# Creamos un constructor de grafo con MessagesState
# MessagesState mantiene automáticamente el historial de mensajes
graph_builder = StateGraph(MessagesState)

# Añadimos los tres nodos principales al grafo:
# 1. query_or_respond: decide si necesita recuperar información
graph_builder.add_node(query_or_respond)
# 2. tools: ejecuta la herramienta de recuperación
graph_builder.add_node(tools)
# 3. generate: genera la respuesta final con el contexto
graph_builder.add_node(generate)

# Definimos el punto de entrada del grafo
# Todas las conversaciones comienzan con query_or_respond
graph_builder.set_entry_point("query_or_respond")

# Añadimos un edge condicional desde query_or_respond
# tools_condition examina si el mensaje de IA contiene llamadas a herramientas:
# - Si hay llamadas a herramientas -> va a "tools"
# - Si no hay llamadas a herramientas -> va a END (termina)
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)

# Después de ejecutar las herramientas, siempre vamos a "generate"
# Este edge conecta la recuperación con la generación de respuesta
graph_builder.add_edge("tools", "generate")

# Después de generar la respuesta, terminamos
# END indica que el flujo ha completado
graph_builder.add_edge("generate", END)

# Creamos el gestor de memoria para persistir el estado entre conversaciones
# MemorySaver guarda checkpoints del estado completo (incluyendo mensajes)
memory = MemorySaver()

# Compilamos el grafo con el checkpointer
# Esto habilita la memoria persistente entre llamadas
graph = graph_builder.compile(checkpointer=memory)

# Mostramos el grafo compilado
graph

In [ ]:
# Especificamos un ID para el hilo de conversación
# El thread_id identifica de manera única esta conversación
# Permite recuperar el historial de mensajes en futuras llamadas
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
# Mensaje de entrada del usuario
input_message = "Hola"

# Ejecutamos el grafo con streaming
# stream() genera valores intermedios mientras el grafo se ejecuta
for step in graph.stream(
    # Creamos el estado inicial con un mensaje del usuario
    {"messages": [{"role": "user", "content": input_message}]},
    # stream_mode="values" retorna el estado completo en cada paso
    stream_mode="values",
    # Pasamos la configuración con el thread_id para memoria persistente
    config=config,
):
    # Imprimimos el último mensaje de cada paso con formato
    # pretty_print() muestra el mensaje con colores y formato legible
    step["messages"][-1].pretty_print()

In [ ]:
# Probamos una búsqueda de similitud directa en el vector store
# Esto nos muestra qué documentos se recuperarían para esta consulta
vector_store.similarity_search("¿Qué es la Descomposición de Tareas?")

In [ ]:
# Segunda interacción: pregunta sobre Task Decomposition
input_message = "¿Qué es la Descomposición de Tareas?"

# Ejecutamos el grafo nuevamente con la misma configuración
# Como usamos el mismo thread_id, el grafo tiene acceso al historial previo
for step in graph.stream(
    # Nuevo mensaje del usuario
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    # Mismo thread_id = misma conversación
    config=config,
):
    # Mostramos cada mensaje generado durante el flujo
    step["messages"][-1].pretty_print()

In [ ]:
# Tercera interacción: pregunta de seguimiento que requiere contexto
# "hacerlo" se refiere a "Task Decomposition" de la pregunta anterior
input_message = "¿Puedes buscar algunas formas comunes de hacerlo?"

# Esta pregunta demuestra la memoria persistente del sistema:
# El agente entiende que "hacerlo" se refiere a Task Decomposition
# porque tiene acceso al historial completo de la conversación
for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
### Historial de Conversación

# Recuperamos el historial completo de la conversación desde el estado guardado
# get_state() recupera el último checkpoint guardado para este thread_id
chat_history = graph.get_state(config).values["messages"]

# Iteramos sobre todos los mensajes e imprimimos cada uno con formato
# Esto muestra la conversación completa incluyendo:
# - Mensajes del usuario (Human)
# - Respuestas de IA (AI)
# - Llamadas a herramientas (Tool calls)
# - Resultados de herramientas (Tool)
for message in chat_history:
    message.pretty_print()

### Arquitectura de Agente ReAct - Memoria Persistente

En esta sección usamos la función preconfigurada `create_react_agent` que implementa el patrón ReAct (Reasoning + Acting) de manera simplificada.

In [ ]:
# Mostramos la herramienta de recuperación que definimos anteriormente
# Esta es la misma herramienta que usaremos con el agente ReAct
retrieve

In [ ]:
# Importamos la función helper para crear agentes ReAct
from langgraph.prebuilt import create_react_agent

# Creamos un nuevo gestor de memoria para este agente
memory = MemorySaver()

# Creamos un agente ReAct preconfigurado
# create_react_agent es una función helper que construye automáticamente:
# 1. Un grafo con el patrón ReAct (razonamiento + acción)
# 2. Manejo de herramientas integrado
# 3. Gestión de memoria persistente
agent_executor = create_react_agent(
    llm,              # El modelo de lenguaje
    [retrieve],       # Lista de herramientas disponibles
    checkpointer=memory  # Gestor de checkpoints para memoria persistente
)

In [ ]:
# Mostramos la configuración del agente ejecutor
# Esto muestra la estructura del grafo compilado con todos sus nodos
agent_executor

In [ ]:
# Configuramos un nuevo thread_id para una conversación diferente
# Este es independiente del thread_id "abc123" que usamos antes
config = {"configurable": {"thread_id": "def234"}}

In [ ]:
# Mensaje de prueba que incluye múltiples tareas
# Este mensaje prueba la capacidad del agente para:
# 1. Responder una pregunta
# 2. Realizar una acción de seguimiento basada en la respuesta
input_message = (
    "¿Cuál es el método estándar para la Descomposición de Tareas?\n\n"
    "Una vez que obtengas la respuesta, busca extensiones comunes de ese método."
)

In [ ]:
# Ejecutamos el agente ReAct con streaming

# Iteramos sobre los eventos generados por el agente
for event in agent_executor.stream(
    # Estado inicial con el mensaje del usuario
    {"messages": [{"role": "user", "content": input_message}]},
    # stream_mode="values" retorna el estado completo en cada paso
    stream_mode="values",
    # Configuración con el thread_id para memoria persistente
    config=config
):
    # Imprimimos el último mensaje de cada evento
    # Esto mostrará:
    # 1. El mensaje del usuario
    # 2. Las llamadas a herramientas que hace el agente
    # 3. Los resultados de las herramientas
    # 4. La respuesta final del agente
    event["messages"][-1].pretty_print()